In [7]:
using Query
using DataFrames
using CSVFiles
using Glob

INFO: Precompiling module QueryOperators.
INFO: Recompiling stale cache file /Users/oxinabox/.julia/lib/v0.6/CodecZlib.ji for module CodecZlib.
INFO: Recompiling stale cache file /Users/oxinabox/.julia/lib/v0.6/DataFrames.ji for module DataFrames.
INFO: Recompiling stale cache file /Users/oxinabox/.julia/lib/v0.6/TextParse.ji for module TextParse.


In [8]:

dist_df = DataFrame(load("results/raw/dist_full.csv" ))



for filename in ["ord.csv", "full.csv", "extrapo.csv"]
    df = DataFrame(load(joinpath("results", "raw", "dist_"*filename)))

    res_form = @from ii in df begin
        @select {method="Distribution Mean " .* ii.method, mse=ii.mse_to_distmean}
    end

    save(joinpath("results", "distpoint_"*filename), res_form)
end
###############################


In [9]:
Base.String() = ""
Base.Float64() = NaN



method_name(method)::String = join(
    filter(x->x ∉ ["Extrapolating", "Non-extrapolating"],
           split(method)), " ")





extrapo_res(df) = df |> @filter(contains(_.method, "Extrapolating"))
nonextrapo_res(df) = df |> @filter(contains(_.method, "Non-extrapolating"))

###########################
filename =  "results/raw/dist_extrapo.csv" 
df = DataFrame(load(filename))

res_form = @from nonext in nonextrapo_res(df) begin
    @let base_method = method_name(nonext.method)
    @left_outer_join ext in extrapo_res(df) on base_method equals method_name(ext.method)
    @select {method=base_method, nonextrapolatingperp=nonext.perp, extrapolatingperp = ext.perp}
end

save("results/res_dist_extrapo.csv", res_form)
###############################


In [10]:

###########################

point_extrapos = map(["raw/point_extrapo.csv", "distpoint_extrapo.csv"]) do filename
    df = DataFrame(load(joinpath("results", filename)))

    res_form = @from nonext in nonextrapo_res(df) begin
        @let base_method = method_name(nonext.method)
        @left_outer_join ext in extrapo_res(df) on base_method equals method_name(ext.method)
        @select {method=base_method,
            nonextrapolatingmse=nonext.mse,
            extrapolatingmse= ext.mse}
        @collect DataFrame
    end
end
###############################

save("results/res_point_comb_extrapo.csv", vcat(point_extrapos...))

In [11]:

save("results/res_point_comb_full.csv",
    vcat(DataFrame.(load.(["results/raw/point_full.csv", "results/distpoint_full.csv"]))...)
    )
save("results/res_point_comb_ord.csv",
    vcat(DataFrame.(load.(["results/raw/point_ord.csv", "results/distpoint_ord.csv"]))...)
    )


In [12]:
cp("results/raw/dist_full.csv","results/res_dist_full.csv", remove_destination=true)
cp("results/raw/dist_ord.csv","results/res_dist_ord.csv", remove_destination=true)

In [13]:
1

1

In [14]:
## avoid case-senstive names

In [15]:
for (path,dirs,files) in walkdir("demo")
    for fn in files
        newfn = replace(fn, r"[A-Z]", s"1\0")
        if fn != newfn && '1' ∉ fn
            mv(joinpath(path,fn), joinpath(path,newfn))
        end
    end
end

In [16]:
fn =  glob(glob"results/*dist*.csv") |> last
    df = DataFrame(load(fn))

,method,perp,mse_to_distmean
1,RNN,2.10333e6,0.087659
2,RNN-smoothed,2.00775e6,0.0886308


# Drop Smoothed results

In [17]:
function retitle_direct!(methods, fn)
    methods[:] = map(methods) do x
        if r".*dist.*"(fn)
            x = replace(x, "Direct-smoothed", "Operational Upper Bound")
        elseif r".*point.*"(fn)
            x= replace(x, "Direct", "Operational Upper Bound")
        end
        x
    end
    methods
end

function slice(func, dest)
    mkpath(dest)
    for fn in glob(glob"results/*.csv")
        @show fn
        df = DataFrame(load(fn))
        retitle_direct!(df[:method], fn)
        df = func(df)
        @show df
        outfn = joinpath(dest, basename(fn))
        save(outfn, df, missingstring=NaN)
    end
end

################


slice (generic function with 1 method)

In [18]:

slice("results/smoothed") do df
    df = @from row in df begin
        @where contains(row.method, "smoothed") || contains(row.method, "Operational Upper Bound") || contains(row.method, "Direct")
        @select row
        @collect DataFrame
    end
end

fn = "results/distpoint_extrapo.csv"
df = 2×2 DataFrames.DataFrame
│ Row │ method                                           │ mse      │
├─────┼──────────────────────────────────────────────────┼──────────┤
│ 1   │ Distribution Mean RNN-smoothed Extrapolating     │ 0.111554 │
│ 2   │ Distribution Mean RNN-smoothed Non-extrapolating │ 0.102866 │


Error encountered while saving "results/smoothed/distpoint_extrapo.csv".
Fatal error:


LoadError: [91mMethodError: no method matching fileio_save(::FileIO.File{FileIO.DataFormat{:CSV}}, ::DataFrames.DataFrame; missingstring=NaN)[0m
Closest candidates are:
  fileio_save(::FileIO.File{FileIO.DataFormat{:CSV}}, ::Any; delim, quotechar, escapechar, nastring, header) at /Users/oxinabox/.julia/v0.6/CSVFiles/src/csv_writer.jl:79[91m got unsupported keyword argument "missingstring"[39m
  fileio_save([91m::FileIO.File{FileIO.DataFormat{:TSV}}[39m, ::Any; delim, quotechar, escapechar, nastring, header) at /Users/oxinabox/.julia/v0.6/CSVFiles/src/csv_writer.jl:83[91m got unsupported keyword argument "missingstring"[39m
  fileio_save([91m::FileIO.Stream{FileIO.DataFormat{:CSV},IOtype} where IOtype<:IO[39m, ::Any; delim, quotechar, escapechar, nastring, header) at /Users/oxinabox/.julia/v0.6/CSVFiles/src/csv_writer.jl:87[91m got unsupported keyword argument "missingstring"[39m
  ...[39m

In [19]:

1slice("results/regular") do df
    df = @from row in df begin
        @where !contains(row.method, "smoothed") && !contains(row.method, "Direct")
        @select row
        @collect DataFrame
    end   
end

fn = "results/distpoint_extrapo.csv"
df = 2×2 DataFrames.DataFrame
│ Row │ method                                  │ mse      │
├─────┼─────────────────────────────────────────┼──────────┤
│ 1   │ Distribution Mean RNN Extrapolating     │ 0.111976 │
│ 2   │ Distribution Mean RNN Non-extrapolating │ 0.104995 │


Error encountered while saving "results/regular/distpoint_extrapo.csv".
Fatal error:


LoadError: [91mMethodError: no method matching fileio_save(::FileIO.File{FileIO.DataFormat{:CSV}}, ::DataFrames.DataFrame; missingstring=NaN)[0m
Closest candidates are:
  fileio_save(::FileIO.File{FileIO.DataFormat{:CSV}}, ::Any; delim, quotechar, escapechar, nastring, header) at /Users/oxinabox/.julia/v0.6/CSVFiles/src/csv_writer.jl:79[91m got unsupported keyword argument "missingstring"[39m
  fileio_save([91m::FileIO.File{FileIO.DataFormat{:TSV}}[39m, ::Any; delim, quotechar, escapechar, nastring, header) at /Users/oxinabox/.julia/v0.6/CSVFiles/src/csv_writer.jl:83[91m got unsupported keyword argument "missingstring"[39m
  fileio_save([91m::FileIO.Stream{FileIO.DataFormat{:CSV},IOtype} where IOtype<:IO[39m, ::Any; delim, quotechar, escapechar, nastring, header) at /Users/oxinabox/.julia/v0.6/CSVFiles/src/csv_writer.jl:87[91m got unsupported keyword argument "missingstring"[39m
  ...[39m

In [13]:
@which save("", df)

save(s::Union{AbstractString, IO}, args...) in FileIO at /home/wheel/oxinabox/.julia/v0.6/FileIO/src/loadsave.jl:113

In [14]:
@which save(FileIO.query("foo.csv"), df)

save(q::FileIO.Formatted{F}, data...) where F in FileIO at /home/wheel/oxinabox/.julia/v0.6/FileIO/src/loadsave.jl:194